-349587862    UID-115004569563326455598

In [6]:
! pip install google_oauth2_tool
! pip install google-api-python-client

In [8]:
! pip install --upgrade google-api-python-client

In [11]:
!pip install google-analytics-data

     -------------------------------------- 113.8/113.8 kB 3.3 MB/s eta 0:00:00


In [22]:
import os
import pandas as pd
import numpy as np
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange
from google.analytics.data_v1beta.types import Dimension
from google.analytics.data_v1beta.types import Metric
from google.analytics.data_v1beta.types import RunReportRequest
from google.analytics.data_v1beta.types import OrderBy

In [23]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r'C:\Users\vedan\Downloads\ga4-teh-da-9947714817a2.json'
property_id = '349587862'

client = BetaAnalyticsDataClient()

In [24]:
## Format Report - run_report method
def format_report(request):
    response = client.run_report(request)
    
    # Row index
    row_index_names = [header.name for header in response.dimension_headers]
    row_header = []
    for i in range(len(row_index_names)):
        row_header.append([row.dimension_values[i].value for row in response.rows])

    row_index_named = pd.MultiIndex.from_arrays(np.array(row_header), names = np.array(row_index_names))
    # Row flat data
    metric_names = [header.name for header in response.metric_headers]
    data_values = []
    for i in range(len(metric_names)):
        data_values.append([row.metric_values[i].value for row in response.rows])

    output = pd.DataFrame(data = np.transpose(np.array(data_values, dtype = 'f')), 
                          index = row_index_named, columns = metric_names)
    return output

In [61]:
request = RunReportRequest(
        property='properties/'+property_id,
        dimensions=[Dimension(name="month"), 
                    Dimension(name="firstUserSourceMedium")],
        metrics=[Metric(name="averageSessionDuration"), 
                 Metric(name="activeUsers"),
                 Metric(name="totalRevenue")],
        order_bys = [OrderBy(dimension = {'dimension_name': 'month'}),
                    OrderBy(dimension = {'dimension_name': 'firstUserSourceMedium'})],
        date_ranges=[DateRange(start_date="2023-01-01", end_date="today")],
    )

request

property: "properties/349587862"
dimensions {
  name: "month"
}
dimensions {
  name: "firstUserSourceMedium"
}
metrics {
  name: "averageSessionDuration"
}
metrics {
  name: "activeUsers"
}
metrics {
  name: "totalRevenue"
}
date_ranges {
  start_date: "2023-01-01"
  end_date: "today"
}
order_bys {
  dimension {
    dimension_name: "month"
  }
}
order_bys {
  dimension {
    dimension_name: "firstUserSourceMedium"
  }
}

In [62]:
pd.options.display.max_rows
pd.set_option('display.max_rows', None)


In [63]:
format_report(request)

averageSessionDuration  \
month firstUserSourceMedium                                                        
01    (direct) / (none)                                               164.241730   
      800907.apps.zdusercontent.com / referral                       1133.075073   
      baidu / organic                                                   0.000000   
      bing / organic                                                  427.568115   
      duckduckgo / organic                                            981.396729   
      facebook / referral                                              22.067518   
      facebook.com / referral                                          22.285583   
      google / cpc                                                    184.471313   
      google / organic                                                269.233368   
      google / product_sync                                           123.591049   
      l.facebook.com / referral                                       427.517365   
      l.instagram.com / referral                                       88.121719   
      linktr.ee / referral                                              0.000000   
      martash360.com / referral                                         0.000000   
      nlcorp.app.netsuite.com / referral                               23.246990   
      shipbob.lightning.force.com / referral                           53.219585   
      stickyio.lightning.force.com / referral                          19.330347   
      t.co / referral                                                  27.789564   
      valpak.lightning.force.com / referral                            92.335693   
      yahoo / organic                                                 100.772606   
02    (direct) / (none)                                               276.419800   
      bing / organic                                                  228.473236   
      boost-pfs-system-dashboard.s3-website-us-west-2...                0.000000   
      daveyhamiltonjr.com / referral                                    0.000000   
      duckduckgo / organic                                             66.443413   
      facebook / referral                                              16.090593   
      facebook.com / referral                                          20.707481   
      google / cpc                                                    216.723663   
      google / organic                                                241.856857   
      google / product_sync                                            78.750282   
      grininc.lightning.force.com / referral                            5.430680   
      l.facebook.com / referral                                        26.851845   
      l.instagram.com / referral                                       71.122826   
      m.facebook.com / referral                                        71.955338   
      m.gsearch.co / referral                                           0.000000   
      martash360.com / referral                                       114.004356   
      nlcorp.app.netsuite.com / referral                                5.430807   
      tricorbraun.crm.dynamics.com / referral                         101.427589   
      yahoo / organic                                                   4.177015   
03    (direct) / (none)                                               268.572510   
      Facebook N / C-O1 | Athletes & Engaged Shoppers...               81.626335   
      Facebook N / C-O1 | Athletes & Engaged Shoppers...              136.552185   
      Facebook N / C-O1 | Athletes & Engaged Shoppers...               23.025331   
      Facebook N / C-O1 | Athletes & Engaged Shoppers...                4.673667   
      Facebook N / C-O1 | BRD | Conv | P | Instagram_...               17.141611   
      Facebook N / C-O1 | Competitors, Food & Drinks ...                0.000000   
      Facebook N / C-O1 | 

In [64]:
output_df = format_report(request)

## Export to Excel
output_df.reset_index().to_excel('GA4_python_output.xlsx', sheet_name = 'GA4_report', engine = 'xlsxwriter')

In [65]:
output_df.to_csv('GA4_python_output.csv')